In [1]:
# Construction of dataset

import os, itertools, time, pickle
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp

USE_folder = "/home/vlead/USE"
alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments
        
reference_alignments = load_alignments(alignment_folder)

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()
    
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        return [(a,b,"subclass_of") for (a,b) in self.get_subclasses()]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = self.object_properties
        data_props = self.data_properties
        props = obj_props + data_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        if union_flag == 0:
            return self.triples
        else:
            return self.parse_triples(union_flag = 1, subclass_of = False)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    continue
                if self.extract_ID(level1_class[0]):
                    subclass_pairs.append((level1_class[0], el.parentNode))
                else:
                    level2classes = level1_class[0].getElementsByTagName("owl:Class")
                    
                    subclass_pairs.extend([(elem, el.parentNode) for elem in level2classes if self.extract_ID(elem)])
        return subclass_pairs
        
    def get_subclasses(self):
        return [(self.extract_ID(a), self.extract_ID(b)) for (a,b) in self.subclasses]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        return element_id.split("#")[-1]
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))




In [3]:
# Extracting USE embeddings

ontologies_in_alignment = [l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")]

def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples(subclass_of=False))))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))

inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\S+")
X = vectorizer.fit_transform(inp)
word2idx_tfidf = {word: i for (i, word)  in enumerate(vectorizer.get_feature_names())}
entity2idx_tfidf = {word.split("#")[1]: i for (i, word)  in enumerate(extracted_elems)}


print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

embeds = extractUSEEmbeddings(inp)
embeddings = dict(zip(extracted_elems, embeds))    

Total number of extracted unique classes and properties from entire RA set:  834


In [4]:
# Type storage

types_dict = {}

def get_tfidf_score(word, phrase):
    return np.sum([X[entity2idx_tfidf[phrase]][:,word2idx_tfidf[word]][0,0] for word in parse(phrase)])
    
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()

    for entity in entities:
        types_dict[entity] = {"type": "entity"}
    for prop in props:
        types_dict[prop] = {"type": "property"}


In [5]:
# Combinatorial mapping generation

all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()
    
    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    all_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    

In [6]:
gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]

data = {}
for mapping in all_mappings:
    if mapping in gt_mappings:
        data[(mapping[0], mapping[1])] = True
    else:
        data[(mapping[0], mapping[1])] = False


In [347]:
def greedy_matching():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = OrderedDict()
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))

        print ("F batch size: ", batch_size_f)
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f

            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            print ("Inputs len: ", len(inputs))
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])

            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices], targets[indices]
            inputs = torch.LongTensor(list(zip(*inputs)))
            targets = torch.LongTensor(targets)

            outputs = model(inputs, 1)
            outputs = [el.item() for el in outputs]
    #             print ([(el, targets[i]) for i,el in enumerate(outputs) if targets[i]])
            print ("Targets: ", targets)
    #             write (("Outputs initially: ", str([str(s) for s in outputs])))
    #             outputs /= torch.sum(outputs, dim=1).view(-1, 1)
    #             write (("Outputs Finally: ", str([str(s) for s in outputs])))
    #             outputs = [el[1].item() for el in outputs]


            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[inputs.numpy()[0][idx]]
                ent2 = emb_indexer_inv[inputs.numpy()[1][idx]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])

    #         all_results = OrderedDict(sorted(all_results.items(), key=lambda x: x[0], reverse=True))
    #         filtered_results = dict()
    #         entities_to_assign = set([el[0] for el in list(all_results.keys())])
    #         for pair in all_results:
    #             if pair[0] in entities_to_assign:
    #                 filtered_results[pair] = all_results[pair]
    #                 entities_to_assign.remove(pair[0])
    #         filtered_results = OrderedDict(sorted(filtered_results.items(), key=lambda x: x[1][0], reverse=True))
    #         
    #     min_val = np.min([el[0] for el in list(all_results.values())])
    #         max_val = np.max([el[0] for el in list(all_results.values())])
    #         normalized_results = {}
    #         for key,val in all_results.items():
    #             tmp = (np.array(val[0]) - min_val) / (max_val - min_val)
    #             normalized_results[key] = (tmp[1], val[1])

        optimum_metrics, opt_threshold = [-1000 for i in range(5)], -1000
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.02
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.02
        low_threshold = 0.9
        high_threshold = 1.01
        print ("Low:", low_threshold, "High:", high_threshold)
        for j,threshold in enumerate(np.arange(low_threshold, high_threshold, 0.0001)):
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)

            fn_list = [key for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
            fp_list = [elem for elem in res if not all_results[elem][1]]
            tp_list = [elem for elem in res if all_results[elem][1]]

            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)


            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                continue
            print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)

            if f1score > optimum_metrics[2]:
                optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
                opt_threshold = threshold

        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(*optimum_metrics))
        all_metrics.append((opt_threshold, optimum_metrics))

    print ("Final Results: ", np.mean([el[1] for el in all_metrics], axis=0))
    print ("Best threshold: ", all_metrics[np.argmax([el[1][2] for el in all_metrics])][0])
    return all_results

In [352]:
def write(elem):
    f = open("Logs", "a+")
    if type(elem) == list or type(elem) == tuple:
        string = str("\n".join([str(s) for s in elem]))
    else:
        string = str(elem)
    f.write("\n"+string)
    f.close()
    

class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.num_directions = 1
        self.hidden_dim = 250
        self.embedding_dim = 512
        self.num_layers = 1
        
        self.name_embedding = nn.Embedding(len(embeddings), 512)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.weight.requires_grad = False
        
        self.dropout = dropout
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers)
        self.layer1 = nn.Bilinear(512, 512, 1)
        self.layer2 = nn.Linear(512, 100)
        self.layer3 = nn.Linear(512, 1)

    def forward(self, inputs, epoch):
        results = []
        for i in range(2):
            x = self.name_embedding(inputs[i])
#             op, (ht, ct) = self.lstm(x.unsqueeze(0))
#             x = ht.permute(1,0,2).reshape(-1, self.num_directions*self.hidden_dim)
#             if epoch == "test":
#                 print ("1", x)
            x = self.layer2(x)
#             if epoch == "test":
#                 print ("Linear", x)
#             x = F.dropout(x, p=0.3)
            results.append(x)
#             print ("Embeddings", x)
#         x = results[0] @ results[1].T
#         print (x)
#         x = torch.mean(x, axis=1)
#         if epoch == num_epochs -1:
#             print ("zipped:", list(zip(results[0].detach().numpy(), results[1].detach().numpy())))
#         inp = torch.cat((results[0]-results[1], results[0]+results[1], results[0], results[1]), dim=1)
        x = self.cosine_sim_layer(results[0], results[1])
#         x = self.layer1(results[0], results[1])
#         if epoch == "test":
#             print ("cos sim:", x)
#         x = self.layer1(results[0], results[1])
#         if epoch == num_epochs-1:
#             print (x)
#         x = F.log_softmax(x)
#         inp = results[0]-results[1]
#         print (inp)
#         x = self.layer1(results[0], results[1])
#         print (F.sigmoid(x))
        return x


In [356]:
emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: word for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data(elem_tuple):
    return np.array([emb_indexer[elem] for elem in elem_tuple])

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])
    return inputs, targets
    
data_items = data.items()
np.random.shuffle(list(data_items))
data = OrderedDict(data_items)

# data = OrderedDict(sorted(data.items(), key=lambda x: x[0]))

print ("Number of entities:", len(data))
all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))

all_metrics = []

def subset(data):
    return np.array(data)[np.random.choice(len(data), int(0.9*len(data)), replace=False), :]

for i in list(range(0, len(all_ont_pairs), 3)):
    
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    torch.set_default_dtype(torch.float64)
    
    train_test_split = 0.9

    train_data_t = [key for key in train_data if data[key]]
    train_data_f = [key for key in train_data if not data[key]]
#     train_data_f = train_data_f[:int(len(train_data_t))]
    np.random.shuffle(train_data_f)
    
    lr = 0.001
    num_epochs = 200
    weight_decay = 0.001
    batch_size = 10
    dropout = 0.3
    batch_size = min(batch_size, len(train_data_t))
    num_batches = int(ceil(len(train_data_t)/batch_size))
    batch_size_f = int(ceil(len(train_data_f)/num_batches))
    
    model = SiameseNetwork()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        inputs_pos, targets_pos = generate_input(train_data_t, 1)
        inputs_neg, targets_neg = generate_input(train_data_f, 0)
        
#         indices = np.random.permutation(len(inputs_pos) + len(inputs_neg))
        
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]
        
        indices_pos = np.random.permutation(len(inputs_pos))
        indices_neg = np.random.permutation(len(inputs_neg))

        inputs_pos, targets_pos = inputs_pos[indices_pos], targets_pos[indices_pos]
        inputs_neg, targets_neg = inputs_neg[indices_neg], targets_neg[indices_neg]
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]
        
        
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))
#         targets = np.array(list(targets_pos) + list(targets_neg))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            
            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f

            inputs = np.concatenate((inputs_pos[batch_start: batch_end], inputs_neg[batch_start_f: batch_end_f]))
            targets = np.concatenate((targets_pos[batch_start: batch_end], targets_neg[batch_start_f: batch_end_f]))
#             print (inputs.shape)
#             print (batch_start, batch_end, len(inputs_pos))
            inp_elems = torch.LongTensor(inputs.T)
            targ_elems = torch.DoubleTensor(targets)
#             print (targ_elems)
            optimizer.zero_grad()
            
            outputs = model(inp_elems, epoch)
#             print (outputs)
            loss = F.mse_loss(outputs, targ_elems)
            loss.backward()

            optimizer.step()

            if batch_idx%10 == 0:
#                 print ("Outupts: ", list(zip(outputs.detach().numpy(), targ_elems.detach().numpy())))
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

    model.eval()
    
    test_data_t = [key for key in test_data if data[key]]
    test_data_f = [key for key in test_data if not data[key]]
    
    res = greedy_matching()

#     break

Number of entities: 122893
Epoch: 0 Idx: 0 Loss: 0.3669203997649694
Epoch: 0 Idx: 10 Loss: 0.05248382463125553
Epoch: 0 Idx: 20 Loss: 0.031864414576130866
Epoch: 1 Idx: 0 Loss: 0.027846700141004436
Epoch: 1 Idx: 10 Loss: 0.02188380952629748
Epoch: 1 Idx: 20 Loss: 0.01915662623068194
Epoch: 2 Idx: 0 Loss: 0.015853911077783927
Epoch: 2 Idx: 10 Loss: 0.015837374019042144
Epoch: 2 Idx: 20 Loss: 0.01447448929404626
Epoch: 3 Idx: 0 Loss: 0.014754754386816393
Epoch: 3 Idx: 10 Loss: 0.013236723801575126
Epoch: 3 Idx: 20 Loss: 0.012689831450588416
Epoch: 4 Idx: 0 Loss: 0.0131278504098224
Epoch: 4 Idx: 10 Loss: 0.01154684367927704
Epoch: 4 Idx: 20 Loss: 0.010174584884837306
Epoch: 5 Idx: 0 Loss: 0.009556412107848912
Epoch: 5 Idx: 10 Loss: 0.009379450135617216
Epoch: 5 Idx: 20 Loss: 0.009184878322059028
Epoch: 6 Idx: 0 Loss: 0.008555037969925844
Epoch: 6 Idx: 10 Loss: 0.008053398764476213
Epoch: 6 Idx: 20 Loss: 0.007792902325576926
Epoch: 7 Idx: 0 Loss: 0.0073062833143752014
Epoch: 7 Idx: 10 Loss

Epoch: 61 Idx: 20 Loss: 0.0046137754359557755
Epoch: 62 Idx: 0 Loss: 0.00523556912210918
Epoch: 62 Idx: 10 Loss: 0.005276943747489649
Epoch: 62 Idx: 20 Loss: 0.005395671966918575
Epoch: 63 Idx: 0 Loss: 0.004611695412014171
Epoch: 63 Idx: 10 Loss: 0.004644615258471008
Epoch: 63 Idx: 20 Loss: 0.00428760318143246
Epoch: 64 Idx: 0 Loss: 0.003797120527472149
Epoch: 64 Idx: 10 Loss: 0.004823561761168619
Epoch: 64 Idx: 20 Loss: 0.004166508268236505
Epoch: 65 Idx: 0 Loss: 0.00434113681135325
Epoch: 65 Idx: 10 Loss: 0.004943638914846476
Epoch: 65 Idx: 20 Loss: 0.005126062224052727
Epoch: 66 Idx: 0 Loss: 0.005242923708526492
Epoch: 66 Idx: 10 Loss: 0.004321597804993978
Epoch: 66 Idx: 20 Loss: 0.0041490273750956885
Epoch: 67 Idx: 0 Loss: 0.0038851244956725813
Epoch: 67 Idx: 10 Loss: 0.004330895761338345
Epoch: 67 Idx: 20 Loss: 0.0040948541795563665
Epoch: 68 Idx: 0 Loss: 0.005305570363050572
Epoch: 68 Idx: 10 Loss: 0.004619576068964049
Epoch: 68 Idx: 20 Loss: 0.0050424394724992916
Epoch: 69 Idx: 

Epoch: 122 Idx: 20 Loss: 0.0044563540731021554
Epoch: 123 Idx: 0 Loss: 0.004696659081244712
Epoch: 123 Idx: 10 Loss: 0.004264224186081368
Epoch: 123 Idx: 20 Loss: 0.0044245080059795334
Epoch: 124 Idx: 0 Loss: 0.0041006337324160395
Epoch: 124 Idx: 10 Loss: 0.005363118669084896
Epoch: 124 Idx: 20 Loss: 0.00498918390619061
Epoch: 125 Idx: 0 Loss: 0.004261091872264029
Epoch: 125 Idx: 10 Loss: 0.0056487407465096685
Epoch: 125 Idx: 20 Loss: 0.0046492143333557645
Epoch: 126 Idx: 0 Loss: 0.004466655641167687
Epoch: 126 Idx: 10 Loss: 0.004891833483507098
Epoch: 126 Idx: 20 Loss: 0.004850074359121985
Epoch: 127 Idx: 0 Loss: 0.003941402976227146
Epoch: 127 Idx: 10 Loss: 0.004796718724811929
Epoch: 127 Idx: 20 Loss: 0.005139690216704958
Epoch: 128 Idx: 0 Loss: 0.004601734908842746
Epoch: 128 Idx: 10 Loss: 0.004349272130378555
Epoch: 128 Idx: 20 Loss: 0.004572788885948214
Epoch: 129 Idx: 0 Loss: 0.004110932031951629
Epoch: 129 Idx: 10 Loss: 0.003956663881407098
Epoch: 129 Idx: 20 Loss: 0.0052553614

Epoch: 182 Idx: 20 Loss: 0.0038362755620744054
Epoch: 183 Idx: 0 Loss: 0.003513742576355119
Epoch: 183 Idx: 10 Loss: 0.004187080513041834
Epoch: 183 Idx: 20 Loss: 0.00402604396848766
Epoch: 184 Idx: 0 Loss: 0.0036277865193135718
Epoch: 184 Idx: 10 Loss: 0.003845747261253487
Epoch: 184 Idx: 20 Loss: 0.0034309136833838394
Epoch: 185 Idx: 0 Loss: 0.004016734820400512
Epoch: 185 Idx: 10 Loss: 0.0036688721743358644
Epoch: 185 Idx: 20 Loss: 0.004329544735277642
Epoch: 186 Idx: 0 Loss: 0.004193591498505593
Epoch: 186 Idx: 10 Loss: 0.003778315784904614
Epoch: 186 Idx: 20 Loss: 0.004114981459657948
Epoch: 187 Idx: 0 Loss: 0.0038219953993543754
Epoch: 187 Idx: 10 Loss: 0.003874381112225531
Epoch: 187 Idx: 20 Loss: 0.004716621307466529
Epoch: 188 Idx: 0 Loss: 0.003930879259830767
Epoch: 188 Idx: 10 Loss: 0.004435473660560594
Epoch: 188 Idx: 20 Loss: 0.0038795937484076546
Epoch: 189 Idx: 0 Loss: 0.004043299764411015
Epoch: 189 Idx: 10 Loss: 0.004039171169432373
Epoch: 189 Idx: 20 Loss: 0.004119613

Threshold:  0.9074999999999992 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9075999999999992 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9076999999999992 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9077999999999992 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9078999999999992 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9079999999999991 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9080999999999991 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9081999999999991 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.6349206349206349
Threshold:  0.9082999999999991 0.75 0.39344262295081966 0.5161290322580645 0.4347826086956521 0.

...

Threshold:  0.9590999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9591999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9592999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9593999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9594999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9595999999999935 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9596999999999934 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9597999999999934 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9598999999999934 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.

Threshold:  0.9666999999999927 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9667999999999927 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9668999999999927 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9669999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9670999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9671999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9672999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9673999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9674999999999926 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.

Threshold:  0.9772999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9773999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9774999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9775999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9776999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9777999999999915 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9778999999999914 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9779999999999914 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9780999999999914 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.

Threshold:  0.9876999999999904 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9877999999999904 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9878999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9879999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9880999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9881999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9882999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9883999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.9884999999999903 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.

Best threshold:  0.9014999999999999


In [346]:
d = OrderedDict(sorted(res.items(), key = lambda x:x[1][0], reverse=True))
cleaned_d = {el: d[el] + (cos_sim(embeddings[el[0]], embeddings[el[1]]),) for el in d}
{el: cleaned_d[el] for el in cleaned_d if cleaned_d[el][1]}

{('cmt#Review', 'sigkdd#Review'): (1.0, True, 1.0),
 ('ekaw#Document', 'sigkdd#Document'): (1.0, True, 1.0),
 ('ekaw#Abstract', 'sigkdd#Abstract'): (1.0, True, 1.0),
 ('cmt#Paper', 'sigkdd#Paper'): (1.0, True, 1.0),
 ('cmt#ProgramCommittee', 'sigkdd#Program_Committee'): (1.0, True, 1.0),
 ('ekaw#Invited_Speaker', 'sigkdd#Invited_Speaker'): (1.0, True, 1.0),
 ('confOf#Person', 'iasted#Person'): (1.0, True, 1.0),
 ('cmt#Person', 'sigkdd#Person'): (1.0, True, 1.0),
 ('cmt#ProgramCommitteeMember', 'sigkdd#Program_Committee_member'): (1.0,
  True,
  1.0),
 ('cmt#Document', 'sigkdd#Document'): (1.0, True, 1.0),
 ('ekaw#Conference', 'sigkdd#Conference'): (1.0, True, 1.0),
 ('cmt#Author', 'sigkdd#Author'): (1.0, True, 1.0),
 ('ekaw#Review', 'sigkdd#Review'): (1.0, True, 1.0),
 ('ekaw#Person', 'sigkdd#Person'): (1.0, True, 1.0),
 ('confOf#Author', 'iasted#Author'): (1.0, True, 1.0),
 ('ekaw#Paper', 'sigkdd#Paper'): (1.0, True, 1.0),
 ('cmt#Conference', 'sigkdd#Conference'): (1.0, True, 1.0),
 (

In [276]:
i, j = emb_indexer['conference#Conference_volume'], emb_indexer['ekaw#Demo_Paper']
name_embedding = nn.Embedding(len(embeddings), 512)
name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
name_embedding.require_grad = False
a, b = name_embedding(torch.LongTensor([i, j]))
x = a @ model.layer2.weight.data.T
y = b @ model.layer2.weight.data.T

cos_sim(x.detach().numpy(), y.detach().numpy())

0.994646818129861

In [342]:
indices = np.random.permutation(inputs.shape[0])
inputs, targets = inputs[indices].transpose(1,0,2), targets[indices]
inputs = inputs.transpose(1,0,2)
inputs_elem = inputs.copy()
inputs

ValueError: axes don't match array

In [249]:
i, j

(395, 735)

F batch size:  5780
Inputs len:  5790
Targets:  tensor([0, 0, 0,  ..., 0, 0, 0])
Inputs len:  5790
Targets:  tensor([0, 0, 0,  ..., 0, 0, 0])
Inputs len:  5790
Targets:  tensor([0, 0, 0,  ..., 0, 0, 0])
Inputs len:  5788
Targets:  tensor([0, 0, 0,  ..., 0, 0, 0])
Low: 0.99 High: 1.01
Threshold:  0.99 0.04636785162287481 0.75 0.08733624454148471 0.1858736059479554 0.057077625570776266
Threshold:  0.9901 0.04792332268370607 0.75 0.09009009009009009 0.19083969465648853 0.0589622641509434
Threshold:  0.9902 0.049586776859504134 0.75 0.09302325581395349 0.19607843137254902 0.06097560975609756
Threshold:  0.9903 0.05163511187607573 0.75 0.0966183574879227 0.20242914979757085 0.06345177664974619
Threshold:  0.9904 0.051601423487544484 0.725 0.09634551495016612 0.20083102493074792 0.06337412587412587
Threshold:  0.9904999999999999 0.05272727272727273 0.725 0.09830508474576272 0.20422535211267603 0.06473214285714285
Threshold:  0.9905999999999999 0.054613935969868174 0.725 0.10157618213660245 0

Threshold:  0.9982999999999991 0.75 0.525 0.6176470588235295 0.5585106382978724 0.6907894736842105
Threshold:  0.9983999999999991 0.75 0.525 0.6176470588235295 0.5585106382978724 0.6907894736842105
Threshold:  0.998499999999999 0.75 0.525 0.6176470588235295 0.5585106382978724 0.6907894736842105
Threshold:  0.998599999999999 0.7777777777777778 0.525 0.6268656716417911 0.5614973262032085 0.7094594594594594
Threshold:  0.998699999999999 0.7777777777777778 0.525 0.6268656716417911 0.5614973262032085 0.7094594594594594
Threshold:  0.998799999999999 0.7777777777777778 0.525 0.6268656716417911 0.5614973262032085 0.7094594594594594
Threshold:  0.998899999999999 0.7777777777777778 0.525 0.6268656716417911 0.5614973262032085 0.7094594594594594
Threshold:  0.998999999999999 0.7777777777777778 0.525 0.6268656716417911 0.5614973262032085 0.7094594594594594
Threshold:  0.999099999999999 0.8076923076923077 0.525 0.6363636363636364 0.5645161290322581 0.7291666666666666
Threshold:  0.999199999999999 0.

In [333]:
res

OrderedDict([(('conference#Program_committee', 'edas#CallForReviews'),
              (0.4884483944550273, False)),
             (('conference#has_an_abstract', 'ekaw#organises'),
              (0.5075839046488142, False)),
             (('conference#Organizing_committee', 'ekaw#Conference_Banquet'),
              (0.5022990375323751, False)),
             (('edas#BreakEvent', 'iasted#Registration_fee'),
              (0.5125315822302471, False)),
             (('edas#AccpetIfRoomRating', 'iasted#Book_proceeding'),
              (0.48995178486917845, False)),
             (('edas#ConferenceChair', 'iasted#Hotel_presenter'),
              (0.530616322200941, False)),
             (('edas#isMenuOf', 'iasted#is_situated_in'),
              (0.4980711797336712, False)),
             (('conference#Paid_applicant', 'ekaw#Submitted_Paper'),
              (0.5091246959859644, False)),
             (('conference#Review_preference',
               'edas#ComputerNetworksSecurityTopic'),
          

In [343]:
def generate_data(elem_tuple):
    op = np.array([[emb_indexer[el] for el in neighbours_dicts[elem.split("#")[0]][elem]] for elem in elem_tuple])
    return op

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])
    return inputs, targets

batch_size = min(batch_size, len(test_data_t))
num_batches = int(ceil(len(test_data_t)/batch_size))
batch_size_f = int(ceil(len(test_data_f)/num_batches))

np.random.shuffle(test_data_t)
np.random.shuffle(test_data_f)

for batch_idx in range(num_batches):
    batch_start = batch_idx * batch_size
    batch_end = (batch_idx+1) * batch_size

    batch_start_f = batch_idx * batch_size_f
    batch_end_f = (batch_idx+1) * batch_size_f

    pos_elems = np.array(test_data_t)[batch_start:batch_end]
    neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
    optimizer.zero_grad()

    inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])     
    targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])
    break

NameError: name 'neighbours_dicts' is not defined

In [18]:
F.cosine_similarity(torch.DoubleTensor([[1,2], [3,4]]), torch.DoubleTensor([[10,2], [31,4]]), dim=1)

tensor([0.6139, 0.6974], dtype=torch.float64)

In [27]:
data = """1	USA	3,062,558	+22,366	133,475	+496	1,337,373	1,591,710	15,190	9,251	403	38,461,154	116,185	331,034,037
2	Brazil	1,643,539	+17,468	66,093	+537	1,072,229	505,217	8,318	7,731	311	4,316,284	20,304	212,582,910
3	India	743,481	+23,135	20,653	+479	457,045	265,783	8,944	539	15	10,211,092	7,398	1,380,196,747
4	Russia	694,230	+6,368	10,494	+198	463,880	219,856	2,300	4,757	72	21,537,771	147,584	145,935,642
5	Peru	305,703		10,772		197,619	97,312	1,236	9,270	327	1,800,690	54,604	32,977,406
6	Chile	301,019	+2,462	6,434	+50	268,245	26,340	2,060	15,745	337	1,210,326	63,306	19,118,631
7	Spain	299,210	+341	28,392	+4	N/A	N/A	617	6,400	607	5,734,599	122,652	46,755,120
8	UK	286,349	+581	44,391	+155	N/A	N/A	231	4,218	654	10,651,308	156,886	67,891,886
9	Mexico	261,750	+4,902	31,119	+480	159,657	70,974	378	2,030	241	641,142	4,972	128,951,496
10	Iran	245,688	+2,637	11,931	+200	207,000	26,757	3,270	2,925	142	1,846,793	21,984	84,006,568
11	Italy	241,956	+137	34,899	+30	192,815	14,242	70	4,002	577	5,703,673	94,338	60,460,068
12	Pakistan	234,509	+2,691	4,839	+77	134,957	94,713	2,306	1,061	22	1,445,153	6,541	220,931,964
13	Saudi Arabia	217,108	+3,392	2,017	+49	154,839	60,252	2,268	6,235	58	2,018,657	57,974	34,819,965
14	Turkey	207,897	+1,053	5,260	+19	185,292	17,345	1,152	2,465	62	3,733,218	44,258	84,351,507
15	South Africa	205,721		3,310		97,848	104,563	539	3,468	56	1,864,111	31,426	59,318,236
16	Germany	198,243	+186	9,094	+2	182,700	6,449	298	2,366	109	5,873,563	70,100	83,788,628
17	France	168,810	+475	29,933	+13	77,655	61,222	538	2,586	459	1,384,633	21,212	65,276,107
18	Bangladesh	168,645	+3,027	2,151	+55	78,102	88,392	1	1,024	13	876,480	5,321	164,712,537
19	Colombia	120,281		4,210		50,370	65,701	841	2,364	83	881,190	17,315	50,890,353
20	Canada	106,106	+171	8,708	+15	69,827	27,571	2,128	2,811	231	2,975,711	78,833	37,747,023
21	Qatar	100,945	+600	134	+1	94,903	5,908	154	35,952	48	390,997	139,254	2,807,805
22	China	83,565	+8	4,634		78,528	403	6	58	3	90,410,000	62,814	1,439,323,776
23	Argentina	80,447		1,602	+20	30,095	48,750	676	1,780	35	414,697	9,174	45,201,791
24	Egypt	76,222		3,422		21,238	51,562	41	745	33	135,000	1,319	102,352,798
25	Sweden	73,344	+57	5,447	+11	N/A	N/A	103	7,262	539	520,208	51,504	10,100,272
26	Indonesia	66,226	+1,268	3,309	+68	30,785	32,132		242	12	946,054	3,458	273,563,534
27	Iraq	64,701	+2,426	2,685	+118	36,252	25,764	396	1,608	67	624,420	15,521	40,229,391
28	Belarus	64,003	+199	436	+7	51,902	11,665	89	6,773	46	1,074,240	113,685	9,449,263
29	Ecuador	62,380		4,821		28,872	28,687	288	3,535	273	163,842	9,285	17,646,119
30	Belgium	62,058	+42	9,774	+3	17,122	35,162	27	5,354	843	1,313,064	113,288	11,590,476
31	UAE	52,600	+532	326	+2	41,714	10,560	1	5,317	33	3,544,000	358,271	9,891,960
32	Kuwait	51,245	+601	377	+4	41,515	9,353	159	11,997	88	413,530	96,816	4,271,307
33	Netherlands	50,694	+37	6,132	+4	N/A	N/A	18	2,958	358	616,376	35,971	17,135,554
34	Kazakhstan	49,683	+1,109	264		27,657	21,762	221	2,646	14	1,620,252	86,277	18,779,648
35	Ukraine	49,607	+564	1,283	+21	22,193	26,131	106	1,134	29	731,858	16,737	43,727,995
36	Oman	48,997	+1,262	224	+6	31,000	17,773	127	9,593	44	217,194	42,525	5,107,411
37	Philippines	47,873	+1,540	1,309	+6	12,386	34,178	166	437	12	845,218	7,712	109,599,173
38	Singapore	45,140	+157	26		41,002	4,112	1	7,715	4	757,746	129,506	5,851,040
39	Portugal	44,416	+287	1,629	+9	29,445	13,342	76	4,356	160	1,271,425	124,697	10,196,100
40	Bolivia	40,509	+1,212	1,476	+42	11,929	27,104	71	3,470	126	88,818	7,608	11,674,969
41	Panama	39,334		770		18,036	20,528	146	9,115	178	148,115	34,321	4,315,525
42	Dominican Republic	38,430	+302	821	+17	19,564	18,045	208	3,542	76	169,999	15,669	10,849,442
43	Poland	36,412	+257	1,528	+7	24,238	10,646	73	962	40	1,676,364	44,295	37,845,799
44	Afghanistan	33,384	+194	920	+22	20,179	12,285	31	857	24	76,910	1,975	38,935,009
45	Switzerland	32,369	+54	1,966	+1	29,300	1,103	17	3,740	227	641,982	74,170	8,655,600
46	Israel	31,886	+1,137	342	+8	18,192	13,352	86	3,467	37	1,130,991	122,966	9,197,590
47	Bahrain	29,821		98		25,178	4,545	54	17,525	58	612,096	359,711	1,701,631
48	Romania	29,620	+397	1,799	+31	20,534	7,287	231	1,540	94	783,337	40,725	19,234,834
49	Nigeria	29,286		654		11,828	16,804	7	142	3	152,952	742	206,172,246
50	Armenia	29,285	+349	503	+12	16,907	11,875	10	9,882	170	125,088	42,212	2,963,343
51	Ireland	25,538	+7	1,742	+1	23,364	432	12	5,171	353	473,974	95,975	4,938,531
52	Honduras	24,665	+722	656	+17	2,585	21,424	58	2,490	66	49,308	4,977	9,906,351
53	Guatemala	23,972		981		3,429	19,562	5	1,338	55	64,590	3,605	17,918,789
54	Azerbaijan	21,374	+537	265	+7	12,635	8,474	66	2,108	26	529,875	52,253	10,140,510
55	Ghana	21,077		129		16,070	4,878	6	678	4	313,457	10,086	31,078,435
56	Japan	19,775		977		17,124	1,674	34	156	8	499,898	3,953	126,468,508
57	Austria	18,421	+56	706		16,686	1,029	10	2,045	78	660,149	73,291	9,007,233
58	Moldova	18,141	+235	603	+11	11,241	6,297	391	4,497	149	101,180	25,083	4,033,774
59	Algeria	16,879	+475	968	+9	11,884	4,027	53	385	22			43,858,923
60	Serbia	16,719	+299	330	+13	13,366	3,023	110	1,914	38	455,604	52,149	8,736,632
61	Nepal	16,168	+204	35		7,499	8,634		555	1	574,263	19,706	29,142,042
62	Cameroon	14,916	+2,324	359	+46	11,525	3,032	52	562	14			26,550,049
63	Morocco	14,607	+228	240	+3	10,639	3,728	19	396	7	801,287	21,705	36,916,314
64	S. Korea	13,181	+44	285	+1	11,914	982	15	257	6	1,346,194	26,257	51,270,007
65	Denmark	12,888	+10	609	+2	11,983	296	5	2,225	105	1,157,601	199,843	5,792,556
66	Czechia	12,639	+73	351	+1	7,873	4,415	14	1,180	33	573,446	53,546	10,709,342
67	Ivory Coast	10,966		75		5,384	5,507		416	3	65,888	2,497	26,382,457
68	Uzbekistan	10,587	+225	39	+2	6,690	3,858	36	316	1	1,121,236	33,495	33,474,926
69	Sudan	9,997	+103	622	+6	5,034	4,341		228	14	401	9	43,856,590
70	Norway	8,941	+5	251		8,138	552	3	1,649	46	355,656	65,596	5,421,885
71	Australia	8,755	+169	106		7,455	1,194	10	343	4	2,800,030	109,789	25,503,813
72	Malaysia	8,674	+6	121		8,481	72	2	268	4	816,118	25,211	32,371,249
73	El Salvador	8,307	+280	229	+6	4,929	3,149	234	1,281	35	179,862	27,728	6,486,748
74	Kenya	8,250	+183	167	+3	2,504	5,579	29	153	3	193,455	3,597	53,780,608
75	Kyrgyzstan	8,141	+450	99	+7	2,916	5,126	24	1,248	15	236,893	36,303	6,525,353
76	Senegal	7,547	+69	137	+1	5,023	2,387	34	451	8	86,103	5,142	16,746,376
77	DRC	7,432		182		3,226	4,024		83	2			89,570,521
78	Venezuela	7,411		68		2,100	5,243	19	261	2	1,325,286	46,609	28,434,294
79	Finland	7,262	+5	329		6,700	233	1	1,311	59	255,500	46,112	5,540,878
80	North Macedonia	7,244	+120	351	+5	3,324	3,569	55	3,477	168	69,426	33,324	2,083,372
81	Haiti	6,371	+38	113		1,824	4,434		559	10	13,159	1,154	11,404,332
82	Tajikistan	6,315	+53	53		4,965	1,297		662	6			9,539,280
83	Bulgaria	5,914		250		3,000	2,664	29	851	36	156,351	22,505	6,947,259
84	Ethiopia	5,846		103		2,430	3,313	35	51	0.9	250,604	2,180	114,981,813
85	Gabon	5,743		46		2,574	3,123	11	2,580	21	42,078	18,902	2,226,103
86	Bosnia and Herzegovina	5,621	+163	207	+8	2,693	2,721	4	1,714	63	102,715	31,312	3,280,369
87	Guinea	5,610		34		4,522	1,054	24	427	3	14,407	1,097	13,134,623
88	Costa Rica	5,241		23		1,776	3,442	10	1,029	5	48,487	9,517	5,094,794
89	French Guiana	5,178	+124	21	+1	2,119	3,038	30	17,334	70	8,707	29,147	298,727
90	Mauritania	4,948		133		1,896	2,919	14	1,064	29	13,842	2,977	4,650,341
91	Djibouti	4,878	+56	55		4,621	202		4,936	56	48,170	48,747	988,169
92	Palestine	4,647	+306	18	+1	494	4,135		911	4	122,555	24,020	5,102,269
93	Luxembourg	4,603	+61	110		4,056	437	3	7,352	176	237,755	379,746	626,089
94	Hungary	4,205	+16	589		2,874	742	7	435	61	286,983	29,709	9,659,853
95	CAR	4,033		52		970	3,011	2	835	11	25,185	5,214	4,830,632
96	Greece	3,589	+27	193	+1	1,374	2,022	10	344	19	349,084	33,495	10,421,964
97	Madagascar	3,472	+222	33		1,187	2,252	41	125	1	25,864	934	27,695,216
98	Croatia	3,272	+52	113		2,229	930	5	797	28	86,860	21,161	4,104,709
99	Thailand	3,195		58		3,072	65	1	46	0.8	603,657	8,648	69,803,101
100	Equatorial Guinea	3,071		51		842	2,178		2,189	36	16,000	11,404	1,403,077
101	Albania	3,038	+74	81	+2	1,744	1,213	7	1,056	28	26,239	9,118	2,877,735
102	Somalia	3,015	+9	92		1,096	1,827	2	190	6			15,895,312
103	Mayotte	2,688	+9	34		2,446	208	3	9,851	125	8,800	32,251	272,859
104	Nicaragua	2,519		83		1,238	1,198		380	13			6,625,589
105	Maldives	2,501	+10	12		2,158	331	12	4,626	22	57,611	106,561	540,641
106	Paraguay	2,456		20		1,180	1,256	7	344	3	77,879	10,917	7,133,671
107	Cuba	2,395	+15	86		2,240	69	4	211	8	189,599	16,739	11,326,482
108	Mali	2,331		119		1,547	665		115	6	16,317	806	20,253,296
109	Sri Lanka	2,081	+4	11		1,955	115	1	97	0.5	114,765	5,359	21,414,775
110	South Sudan	2,021		38		333	1,650		181	3	10,824	967	11,195,460
111	Estonia	1,995	+1	69		1,880	46	2	1,504	52	109,979	82,906	1,326,552
112	Lebanon	1,907	+22	36		1,348	523	8	279	5	152,335	22,321	6,824,796
113	Zambia	1,895	+263	42	+12	1,348	505	1	103	2	56,825	3,091	18,386,352
114	Iceland	1,873	+7	10		1,847	16		5,488	29	91,350	267,670	341,278
115	Lithuania	1,844	+3	79		1,547	218	15	678	29	444,407	163,306	2,721,314
116	Malawi	1,818	+76	19		317	1,482	4	95	1.0	17,406	910	19,132,832
117	Guinea-Bissau	1,790		25		760	1,005	5	909	13	1,500	762	1,968,327
118	Slovakia	1,767	+2	28		1,473	266	3	324	5	219,269	40,161	5,459,692
119	Slovenia	1,739	+23	111		1,423	205		836	53	109,596	52,717	2,078,943
120	Congo	1,557		44		501	1,012		282	8			5,518,968
121	Sierra Leone	1,547		62		1,086	399		194	8			7,978,403
122	New Zealand	1,536	+2	22		1,492	22		307	4	416,924	83,350	5,002,100
123	Cabo Verde	1,463		17		722	724		2,631	31	39,485	71,007	556,069
124	Hong Kong	1,300	+14	7		1,161	132	1	173	0.9	347,400	46,333	7,497,896
125	Yemen	1,297	+13	348	+3	591	358		43	12	120	4	29,831,129
126	Tunisia	1,205	+6	50		1,049	106		102	4	73,610	6,227	11,820,333
127	Benin	1,199		21		333	845	1	99	2	56,613	4,669	12,124,990
128	Jordan	1,169	+2	10		969	190	3	115	1.0	434,295	42,559	10,204,565
129	Latvia	1,134	+7	30		1,008	96	1	601	16	160,281	84,999	1,885,691
130	Libya	1,117		34		269	814		163	5	29,175	4,245	6,872,437
131	Rwanda	1,113		3		575	535		86	0.2	163,384	12,612	12,954,270
132	Niger	1,093		68		968	57		45	3	6,581	272	24,206,762
133	Eswatini	1,056	+45	14	+1	570	472	5	910	12	14,247	12,278	1,160,331
134	Mozambique	1,012		8		277	727		32	0.3	34,036	1,089	31,259,506
135	Cyprus	1,004		19		839	146		831	16	164,944	136,600	1,207,495
136	Burkina Faso	1,003	+3	53		860	90		48	3			20,906,123
137	Uganda	971	+18			896	75		21		212,860	4,653	45,744,909
138	Uruguay	960		29		858	73		276	8	72,553	20,885	3,473,939
139	Georgia	958	+5	15		838	105	5	240	4	125,314	31,415	3,989,014
140	Liberia	917	+26	41	+2	394	482		181	8			5,058,522
141	Montenegro	907	+66	17	+3	320	570		1,444	27	14,781	23,534	628,068
142	Chad	872		74		787	11		53	5			16,427,882
143	Andorra	855		52		800	3		11,066	673	3,750	48,533	77,267
144	Jamaica	737	+5	10		591	136		249	3	26,798	9,049	2,961,386
145	Zimbabwe	734		9		197	528		49	0.6	80,654	5,426	14,865,470
146	Sao Tome and Principe	724	+3	13		279	432		3,303	59	1,873	8,545	219,198
147	Diamond Princess	712		13		651	48	4					
148	San Marino	698		42		656	0		20,571	1,238	5,729	168,838	33,932
149	Togo	680		15		450	215	2	82	2	32,950	3,979	8,280,104
150	Malta	673	+1	9		653	11		1,524	20	103,324	233,996	441,564
151	Suriname	614		15	+1	315	284	5	1,047	26	1,244	2,120	586,709
152	Channel Islands	577		47		512	18		3,318	270	21,388	123,000	173,886
153	Réunion	551	+1	3	+1	472	76	3	615	3	35,419	39,556	895,411
154	Namibia	539	+54			25	514	1	212		11,834	4,657	2,541,362
155	Tanzania	509		21		183	305	7	9	0.4			59,741,697
156	Taiwan	449		7		438	4		19	0.3	77,745	3,264	23,817,558
157	Syria	372		14		126	232		21	0.8			17,503,488
158	Vietnam	369				341	28	1	4		275,000	2,825	97,351,365
159	Angola	346		19		108	219	6	11	0.6	10,000	304	32,869,300
160	Mauritius	342		10		330	2		269	8	193,561	152,194	1,271,807
161	Isle of Man	336		24		312	0		3,951	282	6,600	77,611	85,040
162	Myanmar	316		6		245	65		6	0.1	85,689	1,575	54,415,545
163	Botswana	314		1		31	282	1	134	0.4	47,860	20,348	2,352,046
164	Comoros	311		7		266	38		358	8			869,754
165	Guyana	278	+5	15		121	142	10	353	19	2,889	3,673	786,615
166	Martinique	249		14		98	137	5	664	37			375,259
167	Mongolia	225	+5			194	31	2	69		25,137	7,666	3,278,869
168	Eritrea	215				56	159		61				3,547,017
169	Cayman Islands	201		1		194	6		3,058	15	24,637	374,810	65,732
170	Burundi	191		1		118	72		16	0.08	749	63	11,892,090
171	Faeroe Islands	188				188	0		3,847		16,887	345,578	48,866
172	Guadeloupe	184		14		157	13	4	460	35	9,649	24,115	400,125
173	Gibraltar	179				176	3		5,313		14,934	443,264	33,691
174	Bermuda	146		9		137	0	1	2,345	145	12,363	198,529	62,273
175	Brunei	141		3		138	0		322	7	29,841	68,202	437,539
176	Cambodia	141				131	10	1	8		40,521	2,423	16,721,773
177	Trinidad and Tobago	133		8		117	8		95	6	5,558	3,971	1,399,567
178	Monaco	108		4		95	9		2,752	102	16,200	412,781	39,246
179	Aruba	105		3		98	4		983	28	2,762	25,868	106,774
180	Bahamas	104		11		89	4	1	264	28	2,461	6,257	393,298
181	Barbados	98		7		90	1		341	24	8,268	28,770	287,381
182	Lesotho	91				11	80		42		4,331	2,021	2,142,506
183	Liechtenstein	84		1		81	2		2,203	26	900	23,603	38,130
184	Seychelles	81				11	70		824				98,357
185	Bhutan	80				54	26		104		26,060	33,769	771,723
186	Sint Maarten	78		15		63	0		1,819	350	523	12,196	42,883
187	Antigua and Barbuda	70		3		23	44	1	715	31	796	8,127	97,941
188	French Polynesia	62				60	2		221		4,649	16,548	280,934
189	Gambia	61		3		27	31		25	1	3,162	1,308	2,416,981
190	Turks and Caicos	49	+1	2		11	36		1,265	52	790	20,401	38,723
191	Macao	46				45	1		71		4,071	6,268	649,443
192	Saint Martin	43		3		37	3	1	1,112	78	851	22,005	38,673
193	Belize	30		2		19	9		75	5	2,396	6,025	397,699
194	St. Vincent Grenadines	29				29	0		261		926	8,346	110,946
195	Timor-Leste	24				24	0		18		1,568	1,189	1,318,682
196	Curaçao	23		1		19	3		140	6	1,080	6,581	164,104
197	Grenada	23				23	0		204		5,465	48,564	112,532
198	Saint Lucia	22				19	3		120		1,919	10,450	183,641
199	Fiji	21	+2			18	3		23		4,000	4,462	896,546
200	New Caledonia	21				21	0		74		8,645	30,276	285,537
201	Laos	19				19	0		3		18,091	2,486	7,276,805
202	Dominica	18				18	0		250		623	8,654	71,989
203	Saint Kitts and Nevis	16				15	1		301		483	9,078	53,205
204	Falkland Islands	13				13	0		3,736		1,197	343,966	3,480
205	Greenland	13				13	0		229		4,192	73,839	56,772
206	Vatican City	12				12	0		14,981				801
207	Montserrat	11		1		10	0		2,204	200	61	12,220	4,992
208	Papua New Guinea	11				8	3		1		7,147	799	8,948,632
209	Western Sahara	10		1		8	1		17	2			597,434
210	MS Zaandam	9		2			7						
211	British Virgin Islands	8		1		7	0		265	33	439	14,520	30,234
212	Caribbean Netherlands	7				7	0		267		424	16,167	26,227
213	St. Barth	6				6	0		607		507	51,326	9,878
214	Anguilla	3				3	0		200		305	20,327	15,005"""
sorted([(l.split("\t")[1], float(l.split("\t")[-5].replace(",",""))/float(l.split("\t")[-2].replace(",","")))
        for l in data.split("\n")], key = lambda l: l[1], reverse=True)

ValueError: could not convert string to float: 

In [39]:
results = []
for l in data.split("\n"):
    try:
        results.append((l.split("\t")[1], str(float(l.split("\t")[-5].replace(",",""))/float(l.split("\t")[-2].replace(",","")))))
    except:
        continue
results = [l[0] for l in sorted(results, key = lambda l: float(l[1]), reverse=True)]
ranks = {country: i for i,country in enumerate(results)}
ranks_inv = {i:country for i,country in enumerate(results)}

In [50]:
ranks["Russia"]

104